In [1]:
!pip install transformers datasets peft accelerate torch sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
!pip install -U datasets

In [3]:
import torch
import os
device = 0 if torch.cuda.is_available() else -1

In [4]:
from random import shuffle
from datasets import load_dataset
# whole_data = load_dataset("dev7halo/bluehouse-national-petition")
klue_train = load_dataset('klue', 'ynat', split='train')
klue_evaluate = load_dataset('klue', 'ynat', split='validation')
klue_train = klue_train.remove_columns(['guid', 'url', 'date'])
klue_evaluate = klue_evaluate.remove_columns(['guid', 'url', 'date'])
# whole_data = whole_data.remove_columns(['number', '제목', '답변상태', '참여인원', '청원시작','청원마감','답변원고'])
# sample_data = whole_data['train'].train_test_split(test_size=0.05, shuffle=True, seed=37)['test']
# sample_data = sample_data.train_test_split(test_size=20000, shuffle=True, seed=37)
# train_data = sample_data['test']
# valid_test = sample_data['train'].train_test_split(test_size = 1000, shuffle=True, seed=37)
# valid_data = valid_test['train']
# test_data = valid_test['test']
train_data = klue_train.train_test_split(test_size=10000, shuffle=True, seed=37)['test']
test_and_valid_data = klue_evaluate.train_test_split(test_size=1000, shuffle=True, seed=37)
test_data = test_and_valid_data['test']
valid_data = test_and_valid_data['train'].train_test_split(test_size=1000, shuffle=True, seed=37)['test']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model_id='klue/roberta-base'

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        # model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_data.features['label'].names))
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query", "value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="SEQ_CLS"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    # print_gpu_utilization()
    return model, tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
def tokenizing_function(samples):
  return tokenizer(samples['title'], padding='max_length', truncation=True)

train_data = train_data.map(tokenizing_function, batched=True)
valid_data = valid_data.map(tokenizing_function, batched=True)
test_data = test_data.map(tokenizing_function, batched=True)

In [8]:
from sklearn.metrics import accuracy_score
import numpy as np
from torch import cuda
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

In [9]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForMaskedLM
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    learning_rate=2e-5,
    push_to_hub=False,
)

In [34]:
torch.cuda.reset_peak_memory_stats()
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_data.features['label'].names))

#--without lora--#

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

train_out = trainer.train()
eval_data = trainer.evaluate(test_data)
eval_acc = eval_data["eval_accuracy"]
train_time = train_out.metrics["train_runtime"]
max_memory = torch,cuda.max_memory_allocated() / (1024**3)
model.save_pretrained("tmp_model")
model_size = sum(os.path.getsize(os.path.join("tmp_model", f)) for f in os.listdir("tmp_model")) / 1024**2
# model_size = sum(p.numel() for p in model.parameters()) * 4 / (1024**2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-34-3845272dd6cc>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [37]:
#--using lora--#
torch.cuda.reset_peak_memory_stats()
model_lora, tokenizer_lora = load_model_and_tokenizer(model_id, 'lora')
trainer_lora = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    tokenizer=tokenizer_lora,
    compute_metrics=compute_metrics
)
train_out_lora = trainer_lora.train()
eval_data_lora = trainer_lora.evaluate(test_data)
eval_acc_lora = eval_data_lora["eval_accuracy"]
train_time_lora = train_out_lora.metrics["train_runtime"]
max_memory_lora = torch,cuda.max_memory_allocated() / (1024**3)

model_lora.save_pretrained("tmp_model")
model_size_lora = sum(os.path.getsize(os.path.join("tmp_model", f)) for f in os.listdir("tmp_model")) / 1024**2
# model_size_lora = sum(p.numel() for p in model_lora.parameters()) * 4 / (1024**2)


In [38]:
print(f"{eval_acc} | {eval_acc_lora}")
print(f"{train_time} | {train_time_lora}")
print(f"{max_memory[1]}GB | {max_memory_lora[1]}GB")
print(f"{model_size}MB | {model_size_lora}MB")

0.868 | 0.768
947.5343 | 707.2173
2.1715545654296875GB | 2.836458683013916GB
422.01915168762207MB | 425.4302291870117MB
